# MMSBR preprocess step 2

In [3]:
import os
import pickle
import time
import pandas as pd
import numpy as np

meta_dataset = 'Clothing_Shoes_and_Jewelry'
data_path =  './dict/' + meta_dataset +  '/interaction.csv'

data_all = pd.read_csv(data_path)

data_all = data_all.rename(columns={'reviewerID':'sessionID','asin':'itemID', 'unixReviewTime':'time', 'price_level':'priceLevel', 'cate':'category'})
data_all = data_all[['sessionID', 'itemID', 'priceLevel', 'category', 'time']]



# dict = {reviewerID: sessionID}
reviewerID2sessionID = {}
# dict = {asinID: itemID(renumber starting form 1)}
asin2itemID = {}

category2categoryID = {}

sessionNum = 0
itemNum = 0
categoryNum = 0

# renumber the item & session 
for _, row in data_all.iterrows():
    if row['sessionID'] not in reviewerID2sessionID:
        sessionNum += 1
        reviewerID2sessionID[row['sessionID']] = sessionNum
#     删除不包含image 或 text 的item
    if row['itemID'] not in asin2itemID:
        itemNum += 1
        asin2itemID[row['itemID']] = itemNum
    if row['category'] not in category2categoryID:
        categoryNum += 1
        category2categoryID[row['category']] = categoryNum
    
print('#session: ', sessionNum)
print('&item: ', itemNum)
print('#category: ', categoryNum)

def reSession(reviewerID):
    if reviewerID in reviewerID2sessionID:
        return reviewerID2sessionID[reviewerID]
    else:
        print('session is not recorded')
        return 'none'
    
def reItem(asin):
    if asin in asin2itemID:
        return asin2itemID[asin]
    else:
        print('item is not recorded')
        return 'none'
    
def reCate(category):
    if category in category2categoryID:
        return category2categoryID[category]
    else:
        print('category is not recorded')
        return 'none'
    

def priceInt(price):
    return int(price)


data_all['sessionID']=data_all.sessionID.map(reSession)
data_all['itemID']=data_all.itemID.map(reItem)
data_all['priceLevel']=data_all.priceLevel.map(priceInt)
data_all['category']=data_all.category.map(reCate)
data_all = data_all[data_all['itemID'] != 'none']

item_inter_num = pd.DataFrame(data_all.groupby(data_all['itemID']).count())
item_inter_num = item_inter_num.reset_index()[['sessionID', 'itemID']]
item_num=item_inter_num.rename(columns={'sessionID':'item_num'})
data_all = pd.merge(data_all, item_num, how='left', on = 'itemID')

# 删除出现次数少于5次的item
data_all = data_all[data_all['item_num']>4]
data_all = data_all[['sessionID', 'itemID', 'priceLevel', 'category', 'time']]

save_sess_path = './dict/' + meta_dataset +'/session_data.csv'
save_asin2itemID_path = './dict/' + meta_dataset +'/asin2itemID.csv'
asin_list = list(asin2itemID.keys())
itemID_list = list(asin2itemID.values())
asin2itemID_dict = {'asin':asin_list, 'itemID':itemID_list}
data_asin_itemID = pd.DataFrame(asin2itemID_dict)

data_all.to_csv(save_sess_path)
data_asin_itemID.to_csv(save_asin2itemID_path)
print("dataset: ", meta_dataset)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("done")


#session:  229719
&item:  32941
#category:  1437
dataset:  Sports_and_Outdoors
2022-09-28 17:10:03
done
